# Indigo SP INDY rewards Calx

**The below note book is useful to calcuate INDY rewards distributed to depositors of the Stability Pool(SP) of Indigo Protocol. Indigo Protocol currently has 3 iAsset(iBTC, iETH, iUSD) which have their individual stability pools. These SP pools support the liquidations that occur in the protocol and the depositors of these pools are rewarded with ADA as well as INDY rewards. The APR for these rewards is reported on a per epoch basis and is displyed on the official Indigo Protocol dashboard.**

In [29]:
# ! pip install openpyxl 
import calendar, time;
import os
import pandas as pd
import glob
from datetime import date

**The amount of INDY rewards distributed to SP depositors daily is as follows:**

In [ ]:

path: str = os.getcwd() + "\\INDY_Rewards"
    
excel_files = [file for file in os.listdir(path) if file.endswith('.xlsx')]
df = pd.concat((pd.read_excel(os.path.join(path, file), engine='openpyxl') for file in excel_files), ignore_index=True)
reward_df=df.loc[(df['date'] >= '2024-09-22') & (df['date'] <= '2024-09-26')]

columns_to_drop = ['addr','Expiration','AvailableAt']
reward_df = reward_df.drop(columns=columns_to_drop)
reward_df['amount']=reward_df['amount']/1000000
reward_df=reward_df.groupby(['epoch','date','purpose'])

filtered_df1_month=reward_df.sum()

filtered_df1_month = filtered_df1_month.reset_index()
print(filtered_df1_month.to_string())

**The SP balance of individual SP depositors is as follows:**

In [ ]:
import os
import pandas as pd
import time
from datetime import datetime
from typing import Final
import glob

REF_DATE: Final[datetime] = datetime(2022, 11, 27)
EPOCH: Final[int] = 379

def get_difference(date: datetime) -> int:
    difference = date - REF_DATE

    return difference.days

path: str = os.getcwd() + "\\SP_Staking"

if "SP_Staking_Combined" not in os.listdir(os.getcwd()):
    os.mkdir(f"{os.getcwd()}\\SP_Staking_Combined")

timestamps: list[int] = []
days = []

for filename in os.listdir(path):
    filename = filename.replace(".", "-")
    fname: list[str] = filename.split("-")
    timestamps.append(int(fname[-2]))

for timestamp in timestamps:
    full_date = time.strftime("%d %b %Y", time.gmtime(timestamp))
    date = pd.to_datetime(full_date)
    diff = get_difference(date)
    days.append(diff)

for index, file in list(enumerate(os.listdir(path))):
    df = pd.read_csv(f"{path}\\{file}")
    df.insert(0, column="timestamp", value=timestamps[index])
    df.insert(1, column="Date", value=pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(timestamps[index]))))
    df.insert(2, column="epoch", value= 379 + (days[index])//5 )
    df.to_csv(f"{os.getcwd()}\\SP_Staking_Combined\\{timestamps[index]}.csv")
#     print(file)

# Get CSV files list from a folder
path = "SP_Staking_Combined"
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
big_df   = pd.concat(df_list, ignore_index=True)
columns_to_drop = ['owner','opened_at']

# Drop the specified columns
big_df = big_df.drop(columns=columns_to_drop)

big_df.insert(6, column="Actual_iAsset_Staked", value=big_df['iasset_staked']/1000000)
big_df=big_df.drop("Unnamed: 0",axis=1)
print(big_df.to_string())

**The combined total SP pool deposits for the selected dates is as follows:**

In [ ]:
filtered_df2_month = big_df.loc[(big_df['Date'] >= '2024-09-22')
                     & (big_df['Date'] <= '2024-09-26')]

filtered_df2_month=filtered_df2_month.groupby(['epoch','Date','asset'])
filtered_df2_month=filtered_df2_month.sum()
print(filtered_df2_month.to_string())



**The oracle prices of iAssets are obtained from the Coingecko API. They are as follows:**

In [ ]:
from datetime import date
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

########INDY
data1=cg.get_coin_market_chart_by_id(id='indigo-dao-governance-token', vs_currency='usd', days=365)
# pd.read_json(data)
data1.pop('market_caps')
data1.pop('total_volumes')
dates1=[]
price1=[]
for i,j in data1["prices"]:
    new1=str(pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(i/1000))))
    block=new1.split(" ")[0]
    if pd.to_datetime(block)==date.today():
        break
    if block in dates1:
        break
    else:
        dates1.append(block)
    price1.append(j)


dictionary1= {'Date':dates1,'INDY Price':price1}

########BTC
data2=cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=365)
data2.pop('market_caps')
data2.pop('total_volumes')
dates2=[]
price2=[]
for i,j in data2["prices"]:
    new2=str(pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(i/1000))))
    dates2.append(new2.split(" ")[0])
    price2.append(j)

dictionary2= {'Date':dates2,'BTC Price':price2}
# dictionary2

########ETH
data3=cg.get_coin_market_chart_by_id(id='ethereum', vs_currency='usd', days=365)
data3.pop('market_caps')
data3.pop('total_volumes')
dates3=[]
price3=[]
for i,j in data3["prices"]:
    new3=str(pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(i/1000))))
    dates3.append(new3.split(" ")[0])
    price3.append(j)
dictionary3= {'Date':dates3,'ETH Price':price3}
# dictionary3

########iUSD
data4=cg.get_coin_market_chart_by_id(id='tether', vs_currency='usd', days=365)
# pd.read_json(data)
data4.pop('market_caps')
data4.pop('total_volumes')
dates4=[]
price4=[]
for i,j in data4["prices"]:
    new4=str(pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(i/1000))))
    date=new4.split(" ")[0]
    if date in dates4:
        continue
    dates4.append(date)
    price4.append(j)
dictionary4= {'Date':dates4,'BTC Price':price4}

##################logic_BTC
btc_price=[]
for i in dictionary1["Date"]:
#     print(i)
    for j in set(dates2):
        if i==j:
            btc_price.append(price2[dates2.index(j)])
# len(btc_price)
# ##################logic_ETH
eth_price=[]
for i in dictionary1["Date"]:
    for j in set(dates3):
        if i==j:
            eth_price.append(price3[dates3.index(j)])
# eth_price
# ##################logic_USDT
usd_price=[]
for i in dictionary1["Date"]:
    for j in set(dates4):
        if i==j:
            usd_price.append(price4[dates4.index(j)])
# len(usd_price)


dictionary_combo= {'Date':dates1,'INDY_Price':price1, 'iBTC_Price':btc_price, 'iETH_Price':eth_price, 'iUSD_Price':usd_price}
df1 = pd.DataFrame(dictionary_combo)
print(df1.to_string())

**The APR for iBTC is calculated below:**

In [ ]:
# iBTC
arr1=filtered_df1_month.date.unique()
ent2=[]
for ent in arr1:
    ent=ent.strftime('%Y-%m-%d')
    ent=ent.split(" ")[0]
    ent2.append(ent)
#     print(ent)
print("Dates: "+ str(ent2))
BTC=[]
INDY_price=[]
asset_price=[]
APR=[]

for i in ent2:
    date=i
    # print(i)

    gk1=filtered_df2_month.groupby(['Date','asset'])
    place1=gk1.get_group((date,'iBTC')).sum()
    s11=place1.Actual_iAsset_Staked 
    df2= df1[df1['Date'] == date]
    asset=df2.iBTC_Price.tolist()[0]
    INDY=df2.INDY_Price.tolist()[0]
    asset_price.append(asset)
    INDY_price.append(INDY)
    BTC.append(s11)
    

filtered_df1_month_r=filtered_df1_month[filtered_df1_month["purpose"] == "SP reward for iBTC"]
gk=filtered_df1_month_r.drop(columns=["purpose"])
gk1=gk.groupby(["date"])["amount"].apply(list)
gk1
actual_amount = [];
for i in gk1:
    actual_amount.append(sum(i))

print("asset_staked:"+ str(BTC))
print("asset_price:"+str(asset_price))
print("INDY_price:"+str(INDY_price))
print("INDY_reward:"+str(actual_amount))

numerator=[x * y for x, y in zip(INDY_price, actual_amount)]
print(numerator)
denominator=[x * y for x, y in zip(asset_price, BTC)]
print(denominator)

result = [(x / y)*100*365 if y != 0 else float('inf') for x, y in zip(numerator, denominator)]
print(result)
print("iBTC Mean APR:"+str(sum(result)/len(result)))

**The APR for iETH is calculated below:**

In [ ]:
# ##for iETH  SP APY% for a month
arr1=filtered_df1_month.date.unique()
ent2=[]
for ent in arr1:
    ent=ent.strftime('%Y-%m-%d')
    ent=ent.split(" ")[0]
    ent2.append(ent)
#     print(ent)
print("Dates: "+ str(ent2))
ETH=[]
INDY_price=[]
asset_price=[]
APR=[]

for i in ent2:
    date=i
    

    gk1=filtered_df2_month.groupby(['Date','asset'])
    place1=gk1.get_group((date,'iETH')).sum()
    s11=place1.Actual_iAsset_Staked 
    df2= df1[df1['Date'] == date]
    asset=df2.iETH_Price.tolist()[0]
    INDY=df2.INDY_Price.tolist()[0]
    asset_price.append(asset)
    INDY_price.append(INDY)
    ETH.append(s11)
    

filtered_df1_month_r=filtered_df1_month[filtered_df1_month["purpose"] == "SP reward for iETH"]
gk=filtered_df1_month_r.drop(columns=["purpose"])
gk1=gk.groupby(["date"])["amount"].apply(list)
gk1
actual_amount = [];

for i in gk1:
    actual_amount.append(sum(i))

print("asset_staked:"+ str(ETH))
print("asset_price:"+str(asset_price))
print("INDY_price:"+str(INDY_price))
print("INDY_reward:"+str(actual_amount))

numerator=[x * y for x, y in zip(INDY_price, actual_amount)]
print(numerator)
denominator=[x * y for x, y in zip(asset_price, ETH)]
print(denominator)

result = [(x / y)*100*365 if y != 0 else float('inf') for x, y in zip(numerator, denominator)]
print(result)
print("iETH Mean APR:"+str(sum(result)/len(result)))

**The APR for iUSD is calculated below:**

In [ ]:
# for iUSD  SP APY% for a month
arr1=filtered_df1_month.date.unique()
ent2=[]
for ent in arr1:
    ent=ent.strftime('%Y-%m-%d')
    ent=ent.split(" ")[0]
    ent2.append(ent)
#     print(ent)
print("Dates: "+ str(ent2))
USD=[]
INDY_price=[]
asset_price=[]
APR=[]

for i in ent2:
    date=i
    

    gk1=filtered_df2_month.groupby(['Date','asset'])
    place1=gk1.get_group((date,'iUSD')).sum()#########1#####change1
    s11=place1.Actual_iAsset_Staked 
    df2= df1[df1['Date'] == date]
    asset=df2.iUSD_Price.tolist()[0]#####change2
    INDY=df2.INDY_Price.tolist()[0]
    asset_price.append(asset)
    INDY_price.append(INDY)
    USD.append(s11)
    

filtered_df1_month_r=filtered_df1_month[filtered_df1_month["purpose"] == "SP reward for iUSD"]
gk=filtered_df1_month_r.drop(columns=["purpose"])
gk1=gk.groupby(["date"])["amount"].apply(list)
gk1
actual_amount = [];
for i in gk1:
    actual_amount.append(sum(i))

print("asset_staked:"+ str(USD))
print("asset_price:"+str(asset_price))
print("INDY_price:"+str(INDY_price))
print("INDY_reward:"+str(actual_amount))

numerator=[x * y for x, y in zip(INDY_price, actual_amount)]
print(numerator)
denominator=[x * y for x, y in zip(asset_price, USD)]
print(denominator)

result = [(x / y)*100*365 if y != 0 else float('inf') for x, y in zip(numerator, denominator)]
print(result)
print("iUSD Mean APR:"+str(sum(result)/len(result)))

**The APRs are as follows:**

iBTC: 5.25%

iETH: 10.52%

iUSD: 12.98%